# Телеком: исследование оттока клиентов


In [1]:
# Импорт библиотек

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np


from sklearn.preprocessing import StandardScaler 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score as roc_auc
from sklearn.metrics import accuracy_score as acc_score

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Предобработка данных

In [3]:
try: 
    df_contract = pd.read_csv('D:\\Start\\Data scientist 2.0\\FINAL BOSS\\df\\final_provider\\contract.csv')
except FileNotFoundError:
    df_contract = pd.read_csv('/datasets/final_provider/contract.csv')

In [4]:
print(df_contract.info())
df_contract.head(11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
5,9305-CDSKC,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,99.65,820.5
6,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4
7,6713-OKOMC,2019-04-01,No,Month-to-month,No,Mailed check,29.75,301.9
8,7892-POOKP,2017-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,104.80,3046.05
9,6388-TABGU,2014-12-01,No,One year,No,Bank transfer (automatic),56.15,3487.95


In [5]:
try: 
    df_personal = pd.read_csv('D:\\Start\\Data scientist 2.0\\FINAL BOSS\\df\\final_provider\\personal.csv')
except FileNotFoundError:
    df_contract = pd.read_csv('/datasets/final_provider/personal.csv')

In [6]:
print(df_personal.info())
df_personal.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
None


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [7]:
try:
    df_internet = pd.read_csv('D:\\Start\\Data scientist 2.0\\FINAL BOSS\\df\\final_provider\\internet.csv')
except FileNotFoundError:
    df_contract = pd.read_csv('/datasets/final_provider/internet.csv')

In [8]:
print(df_internet.info())
df_internet.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
None


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


In [9]:
try: 
    df_phone = pd.read_csv('D:\\Start\\Data scientist 2.0\\FINAL BOSS\\df\\final_provider\\phone.csv')
except FileNotFoundError:
    df_contract = pd.read_csv('/datasets/final_provider/phone.csv')

In [10]:
print(df_phone.info())
df_phone.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


Очень не хватает описания данных, что иименно хранится в столбцах, какие бывают варианты значений категориальных данных.

Пропусков в данных нет. В таблице контрактов столбец TotalCharges не того типа данных. Исправим это сразу, проверив предварительно какие там значения имеются.

In [11]:
df_contract['TotalCharges'].value_counts()

          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

Кажется, кто-то забыл ввести данные. Посмотрим на эти строки.

In [12]:
df_contract.loc[df_contract['TotalCharges'] == ' ']

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
488,4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,
753,3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,
936,5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,
1082,4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,
1340,1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,
3331,7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,
3826,3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,
4380,2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,
5218,2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,
6670,4075-WKNIU,2020-02-01,No,Two year,No,Mailed check,73.35,


Все пользователи с пропусками новые. Приравняем total charges к monthly charges.

In [13]:
df_contract.sort_values(by = 'BeginDate').tail(30)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
6499,2676-ISHSF,2020-01-01,No,Month-to-month,No,Mailed check,20.30,20.3
3202,5501-TVMGM,2020-01-01,No,Month-to-month,Yes,Electronic check,55.25,55.25
3258,7365-BVCJH,2020-01-01,No,Month-to-month,Yes,Electronic check,24.40,24.4
6449,7601-WFVZV,2020-01-01,No,Month-to-month,No,Mailed check,19.55,19.55
181,6582-OIVSP,2020-01-01,No,Month-to-month,No,Mailed check,45.30,45.3
3312,9122-UMROB,2020-01-01,No,Month-to-month,No,Mailed check,20.30,20.3
3313,4232-JGKIY,2020-01-01,No,Month-to-month,No,Electronic check,19.20,19.2
3324,5043-TRZWM,2020-01-01,No,Month-to-month,No,Electronic check,75.55,75.55
3328,1640-PLFMP,2020-01-01,No,Month-to-month,Yes,Electronic check,70.25,70.25
1168,5562-BETPV,2020-01-01,No,One year,No,Mailed check,19.65,19.65


In [14]:
for i in range(len(df_contract)):
    if df_contract.loc[i, 'TotalCharges'] == ' ':
        df_contract.loc[i, 'TotalCharges'] = df_contract.loc[i, 'MonthlyCharges']

In [15]:
# Теперь можно менять тип столбца

df_contract['TotalCharges'] = df_contract['TotalCharges'].astype(float)

### Contract

Разберем детально таблицы по очереди, для понимания того, какая информация в них есть.

In [16]:
# Попробуем привести столбец с датой к типу datetime.
df_contract['BeginDate'] = pd.to_datetime(df_contract['BeginDate'])

In [17]:
# Поищем неявные пропуски в EndDate
df_contract['EndDate'].unique()

array(['No', '2019-12-01 00:00:00', '2019-11-01 00:00:00',
       '2019-10-01 00:00:00', '2020-01-01 00:00:00'], dtype=object)

In [18]:
# Посмотрим на возможные значения Type
df_contract['Type'].value_counts()

Month-to-month    3875
Two year          1695
One year          1473
Name: Type, dtype: int64

In [19]:
df_contract['PaperlessBilling'].value_counts()

Yes    4171
No     2872
Name: PaperlessBilling, dtype: int64

In [20]:
df_contract['PaymentMethod'].value_counts()

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64

In [21]:
df_contract['MonthlyCharges'].describe()

count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
25%        35.500000
50%        70.350000
75%        89.850000
max       118.750000
Name: MonthlyCharges, dtype: float64

In [22]:
df_contract['TotalCharges'].describe()

count    7043.000000
mean     2279.798992
std      2266.730170
min        18.800000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64

Неужели кто-то платит по 8,6 тысяч долларов за связь? В любом случае данные почти готовы. Осталось проверить на дубликаты.

In [23]:
df_contract.duplicated().sum()

0

In [24]:
# И ещё одним неявным способом

df_contract['customerID'].value_counts()

7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: customerID, Length: 7043, dtype: int64

ID пользователя действительно уникальны, дубликатов нет. Таблица готова к анализу.

### Personal

In [25]:
df_personal['customerID'].value_counts()

7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: customerID, Length: 7043, dtype: int64

Все Id уникальны.

In [26]:
df_personal['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [27]:
df_personal['SeniorCitizen'].value_counts()

0    5901
1    1142
Name: SeniorCitizen, dtype: int64

In [28]:
df_personal['Partner'].value_counts()

No     3641
Yes    3402
Name: Partner, dtype: int64

In [29]:
df_personal['Dependents'].value_counts()

No     4933
Yes    2110
Name: Dependents, dtype: int64

Аномальных небинарных значений не выявлено.

У нас примерно поровну мужчин и женщин, почти поровну клиентов в браке и холостых. Пожилых людей и клиентов, имеющих иждевенцев, значительно меньше.

### Internet

In [30]:
df_internet['customerID'].value_counts()

7590-VHVEG    1
5181-OABFK    1
9356-AXGMP    1
5176-OLSKT    1
7508-DQAKK    1
             ..
1104-FEJAM    1
2972-YDYUW    1
4238-JSSWH    1
0869-PAPRP    1
3186-AJIEK    1
Name: customerID, Length: 5517, dtype: int64

In [31]:
df_internet['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
Name: InternetService, dtype: int64

In [32]:
df_internet['OnlineSecurity'].value_counts()

No     3498
Yes    2019
Name: OnlineSecurity, dtype: int64

In [33]:
df_internet['OnlineBackup'].value_counts()

No     3088
Yes    2429
Name: OnlineBackup, dtype: int64

In [34]:
df_internet['DeviceProtection'].value_counts()

No     3095
Yes    2422
Name: DeviceProtection, dtype: int64

In [35]:
df_internet['TechSupport'].value_counts()

No     3473
Yes    2044
Name: TechSupport, dtype: int64

In [36]:
df_internet['StreamingTV'].value_counts()

No     2810
Yes    2707
Name: StreamingTV, dtype: int64

In [37]:
df_internet['StreamingMovies'].value_counts()

No     2785
Yes    2732
Name: StreamingMovies, dtype: int64

Интересно, что во всех случаях примерно половина клиентов пользуется одной из дополнительных услуг. Данные чистые-шелковистые.

### Phone

In [38]:
df_phone.duplicated().sum()

0

In [39]:
df_phone['MultipleLines'].value_counts()

No     3390
Yes    2971
Name: MultipleLines, dtype: int64

Здесь с данными тоже все в порядке. Можно переходить к следующему этапу.

## Подготовка данных к обучению моделей

### Target

In [40]:
target = df_contract['EndDate']
target.head(5)

0                     No
1                     No
2    2019-12-01 00:00:00
3                     No
4    2019-11-01 00:00:00
Name: EndDate, dtype: object

In [41]:
target = target.apply(lambda x: 0 if x == 'No' else 1)

In [42]:
target = target.astype('int')
target.head(5)

0    0
1    0
2    1
3    0
4    1
Name: EndDate, dtype: int32

Целевой признак готов для обучения модели.

### Features

In [43]:
features_row = df_contract.drop(['EndDate'], axis=1)

In [44]:
# Теперь будем присоединять другие 3 таблицы

features_row = features_row.merge(df_personal, on='customerID', how='left').merge(
    df_internet, on='customerID', how='left').merge(df_phone, on='customerID', how='left')

In [45]:
print(features_row.info())
features_row.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   Type              7043 non-null   object        
 3   PaperlessBilling  7043 non-null   object        
 4   PaymentMethod     7043 non-null   object        
 5   MonthlyCharges    7043 non-null   float64       
 6   TotalCharges      7043 non-null   float64       
 7   gender            7043 non-null   object        
 8   SeniorCitizen     7043 non-null   int64         
 9   Partner           7043 non-null   object        
 10  Dependents        7043 non-null   object        
 11  InternetService   5517 non-null   object        
 12  OnlineSecurity    5517 non-null   object        
 13  OnlineBackup      5517 non-null   object        
 14  DeviceProtection  5517 n

,customerID,BeginDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,One year,No,Mailed check,56.95,1889.50,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No


Избавимся от ненужных столбцов. По некоторым столбцам можно однозначно сказать, что они не имеют значения. Например customerID, PaymentMethod. Вряд ли клиенты уходят от компании из-за некрасивого айди или способа оплаты.

In [46]:
features_row.drop(['customerID', 'PaymentMethod'], axis=1, inplace=True)

BeginDate может быть важной информацией. Точнее, не дата регистрации, а сколько дней прошло с начала оказания услуг. Информация о договорах актуальна на 1 февраля 2020.

In [47]:
actual_date = datetime(2020, 2, 1)
Days = pd.Series([])
Days = actual_date - features_row['BeginDate'] 
Days

C:\Users\6145~1\AppData\Local\Temp/ipykernel_12444/3201916522.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Days = pd.Series([])


0        31 days
1      1036 days
2       123 days
3      1371 days
4       153 days
          ...   
7038    730 days
7039   2191 days
7040    337 days
7041    215 days
7042   2010 days
Name: BeginDate, Length: 7043, dtype: timedelta64[ns]

In [48]:
features_row['Days'] = Days.dt.days
features_row.drop('BeginDate', axis=1, inplace=True)
features_row.head(5)

,Type,PaperlessBilling,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines,Days
0,Month-to-month,Yes,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN,31
1,One year,No,56.95,1889.50,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No,1036
2,Month-to-month,Yes,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No,123
3,One year,No,42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN,1371
4,Month-to-month,Yes,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No,153


Теперь у нас образовались пропуски. Посмотрим, в каких столбцах они есть и заполним.

In [49]:
features_row.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Type              7043 non-null   object 
 1   PaperlessBilling  7043 non-null   object 
 2   MonthlyCharges    7043 non-null   float64
 3   TotalCharges      7043 non-null   float64
 4   gender            7043 non-null   object 
 5   SeniorCitizen     7043 non-null   int64  
 6   Partner           7043 non-null   object 
 7   Dependents        7043 non-null   object 
 8   InternetService   5517 non-null   object 
 9   OnlineSecurity    5517 non-null   object 
 10  OnlineBackup      5517 non-null   object 
 11  DeviceProtection  5517 non-null   object 
 12  TechSupport       5517 non-null   object 
 13  StreamingTV       5517 non-null   object 
 14  StreamingMovies   5517 non-null   object 
 15  MultipleLines     6361 non-null   object 
 16  Days              7043 non-null   int64  


In [50]:
# Во всех случаях пропуски можно заменить на No
features_row = features_row.fillna('No')

Важный столбец сохранили, пропуски заполнены. Теперь можно переходить к кодированию категориальных признаков. Прямое кодирование использовано для последующей работы с линейной регрессией.

In [51]:
features_to_dummy = ['Type', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 
                                       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                      'StreamingTV', 'StreamingMovies', 'MultipleLines']

In [52]:
features = pd.get_dummies(features_row[features_to_dummy],drop_first = True)

In [53]:
features[['MonthlyCharges', 'TotalCharges', 'Days']] = features_row[['MonthlyCharges', 'TotalCharges', 'Days']]

In [54]:
pd.set_option('display.max_columns', None)
features.head(5)

,SeniorCitizen,Type_One year,Type_Two year,gender_Male,Partner_Yes,Dependents_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,MultipleLines_Yes,MonthlyCharges,TotalCharges,Days
0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,29.85,29.85,31
1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,56.95,1889.50,1036
2,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,53.85,108.15,123
3,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,42.30,1840.75,1371
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,70.70,151.65,153


Масштабируем данные так же для линейной модели.

In [55]:
scaler = StandardScaler()
features = pd.DataFrame(data = scaler.fit_transform(features), columns = features.columns)

In [56]:
features.head(5)

,SeniorCitizen,Type_One year,Type_Two year,gender_Male,Partner_Yes,Dependents_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,MultipleLines_Yes,MonthlyCharges,TotalCharges,Days
0,-0.439916,-0.514249,-0.562975,-1.009559,1.034530,-0.654012,-0.885660,-0.525927,-0.633933,1.378241,-0.723968,-0.639439,-0.790132,-0.79607,-0.854176,-1.160323,-0.992667,-1.324370
1,-0.439916,1.944582,-0.562975,0.990532,-0.966622,-0.654012,-0.885660,-0.525927,1.577454,-0.725563,1.381277,-0.639439,-0.790132,-0.79607,-0.854176,-0.259629,-0.172198,0.040110
2,-0.439916,-0.514249,-0.562975,0.990532,-0.966622,-0.654012,-0.885660,-0.525927,1.577454,1.378241,-0.723968,-0.639439,-0.790132,-0.79607,-0.854176,-0.362660,-0.958122,-1.199462
3,-0.439916,1.944582,-0.562975,0.990532,-0.966622,-0.654012,-0.885660,-0.525927,1.577454,-0.725563,1.381277,1.563872,-0.790132,-0.79607,-0.854176,-0.746535,-0.193706,0.494937
4,-0.439916,-0.514249,-0.562975,-1.009559,-0.966622,-0.654012,1.129102,-0.525927,-0.633933,-0.725563,-0.723968,-0.639439,-0.790132,-0.79607,-0.854176,0.197365,-0.938930,-1.158732


### Деление на выборки

In [57]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, stratify=target, 
                                                                          random_state=12345)

## Обучение моделей

### RandomForest

Для подбора гиперпараметров используем GridSearch

In [58]:
m_forest = RandomForestClassifier()

In [72]:
params = {'n_estimators': range(95, 250, 5), 'max_depth': range(2,16)}

In [73]:
grid = GridSearchCV(estimator=m_forest, param_grid=params, scoring='roc_auc')

In [75]:
%%time
grid.fit(features_train, target_train)

CPU times: total: 13min 56s
Wall time: 13min 59s


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(2, 16),
                         'n_estimators': range(95, 250, 5)},
             scoring='roc_auc')

In [76]:
grid.best_params_

{'max_depth': 12, 'n_estimators': 175}

In [59]:
m_forest = RandomForestClassifier(n_estimators=175, max_depth=12, criterion='gini')

In [60]:
m_forest.fit(features_train, target_train)

RandomForestClassifier(max_depth=12, n_estimators=175)

In [61]:
roc_auc_forest = cross_val_score(m_forest, features_train, target_train, scoring='roc_auc')
roc_auc_forest = sum(roc_auc_forest) / len(roc_auc_forest)

In [62]:
roc_auc_forest

0.8856213626547561

In [63]:
acc_forest = cross_val_score(m_forest, features_train, target_train, scoring='accuracy')
acc_forest = sum(acc_forest) / len(acc_forest)

In [64]:
acc_forest

0.8411449312136625

Дерево решений показало себя неплохо, но но мне кажется, можно лучше.

### Logistic Regression

In [65]:
m_reg = LogisticRegression()

In [66]:
m_reg.fit(features_train, target_train)

LogisticRegression()

In [67]:
roc_auc_reg = cross_val_score(m_reg, features_train, target_train, scoring='roc_auc')
roc_auc_reg = sum(roc_auc_reg) / len(roc_auc_reg)
roc_auc_reg

0.8376082803979431

In [68]:
acc_reg = cross_val_score(m_reg, features_train, target_train, scoring='accuracy')
acc_reg = sum(acc_reg) / len(acc_reg)
acc_reg

0.7955301882896955

Линейная регрессия справилась совсем плохо. Посмотрим на градиентный бустинг.

### Градиентный бустинг

In [69]:
m_lgb = LGBMClassifier()

In [70]:
m_lgb.fit(features_train, target_train)

LGBMClassifier()

In [71]:
roc_auc_lgb = cross_val_score(m_lgb, features_train, target_train, scoring='roc_auc')
roc_auc_lgb = sum(roc_auc_lgb) / len(roc_auc_lgb)
roc_auc_lgb

0.9281177211457358

In [72]:
acc_lgb = cross_val_score(m_lgb, features_train, target_train, scoring='accuracy')
acc_lgb = sum(acc_lgb) / len(acc_lgb)
acc_lgb

0.882854085336351

In [73]:
params = {'n_estimators': range(160, 350, 5), 'max_depth': range(2,16), 'num_leaves': [40, 200, 5]}

In [74]:
%%time
grid = GridSearchCV(estimator=m_lgb, param_grid=params, scoring='roc_auc')
grid.fit(features_train, target_train)

Wall time: 28min 8s


GridSearchCV(estimator=LGBMClassifier(),
             param_grid={'max_depth': range(2, 16),
                         'n_estimators': range(160, 350, 5),
                         'num_leaves': [40, 200, 5]},
             scoring='roc_auc')

In [75]:
grid.best_params_

{'max_depth': 5, 'n_estimators': 340, 'num_leaves': 40}

In [76]:
m_lgb = LGBMClassifier(max_depth = 5, n_estimators = 340, num_leaves = 40)

In [77]:
m_lgb.fit(features_train, target_train)

LGBMClassifier(max_depth=5, n_estimators=340, num_leaves=40)

In [78]:
roc_auc_lgb = cross_val_score(m_lgb, features_train, target_train, scoring='roc_auc')
roc_auc_lgb = sum(roc_auc_lgb) / len(roc_auc_lgb)
roc_auc_lgb

0.9351889961594795

In [79]:
acc_lgb = cross_val_score(m_lgb, features_train, target_train)
acc_lgb = sum(acc_lgb) / len(acc_lgb)
acc_lgb

0.8897759026384513

Гридсёрч помог улучшить модель градиентного бустинга, но в принципе модель и из коробки работает прекрасно.

На финальный тест отправляется модель LGB, её показатели выше остальных.

## Тест лучших моделей

In [80]:
roc_auc_lgb_test = cross_val_score(m_lgb, features_test, target_test, scoring='roc_auc')
roc_auc_lgb_test = sum(roc_auc_lgb_test) / len(roc_auc_lgb_test)
roc_auc_lgb_test

0.8989540845193019

In [81]:
acc_lgb_test = cross_val_score(m_lgb, features_test, target_test)
acc_lgb_test = sum(acc_lgb_test) / len(acc_lgb_test)
acc_lgb_test

0.8672749299613841

Великолепный результат!

**Модель LGB отправляется в продакшн!**